## Database Optimization

### Performance Analysis

In [ ]:
EXPLAIN ANALYZE
SELECT 
    product_id,
    category,
    customer_id,
    order_date,
    total_revenue,
    -- Running total of orders for each product category
    SUM(total_revenue) OVER (PARTITION BY category ORDER BY order_date 
                      ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS running_total,
    -- Rank of each sale within its category based on total_revenue
    RANK() OVER (PARTITION BY category ORDER BY total_revenue DESC) AS category_rank,
    -- Total orders for each customer
    SUM(total_revenue) OVER (PARTITION BY customer_id) AS total_orders_per_customer,
    -- Average sale total_revenue for each customer
    ROUND(AVG(total_revenue) OVER (PARTITION BY customer_id), 2) AS avg_sale_per_customer,
    -- Number of orders made by each customer
    COUNT(*) OVER (PARTITION BY customer_id) AS orders_count_per_customer,
    -- Difference in orders total_revenue compared to the previous sale for the same customer
    total_revenue - LAG(total_revenue) OVER (PARTITION BY customer_id ORDER BY order_date) AS sales_diff
FROM 
    orders
JOIN 
    products 
USING(product_id)
ORDER BY 
    category, order_date;


### Indexes on frequently used columns

In [ ]:
-- Index on customer_id
CREATE INDEX idx_customer_id ON orders (customer_id);

-- Index on order_date
CREATE INDEX idx_order_date ON orders (order_date);

-- Index on total_revenue
CREATE INDEX idx_total_revenue ON orders (total_revenue);

-- Index on product_id
CREATE INDEX idx_product_id ON orders (product_id);

-- -- Index on customer_id
CREATE INDEX idx_customers_customer_id ON customers (customer_id);

-- Index on product_id
CREATE INDEX idx_products_product_id ON products (product_id);


In [ ]:
-- Verify index creation

-- Show indexes on the orders table
SELECT * FROM pg_indexes WHERE tablename = 'orders';

-- Show indexes on the products table
SELECT * FROM pg_indexes WHERE tablename = 'products';

-- Show indexes on the customers table
SELECT * FROM pg_indexes WHERE tablename = 'customers';


### Profiling tools to monitor performance

In [ ]:
-- Enable the extension
CREATE EXTENSION pg_stat_statements;



In [ ]:
-- View the statistics
SELECT 
    * 
FROM 
    pg_stat_statements 
ORDER BY 
    total_time DESC 
LIMIT 10;

In [ ]:
SELECT 
    query,
    calls,
    total_time,
    mean_time,
    rows
FROM 
    pg_stat_statements
ORDER BY 
    total_time DESC
LIMIT 10;
